In [47]:
import pandas as pd
import sqlite3
from sqlite3 import connect
from sqlalchemy import create_engine
import re
from nltk.stem.snowball import SnowballStemmer

In [8]:
con = sqlite3.connect('/Users/sean/labs/Capstone/TwitterStockAnalysis/sentiment.db')

In [20]:
df = pd.read_sql_query("SELECT * FROM spark_cleaned", con)

In [22]:
df.head(5)

,index,screen_name,date_time,text,retweeted,lang
0,0,Zed18978371,2021-09-04 12:48:31.000000,@NorristownN @RolandStautz Yup - I’m the crazy...,0,en
1,1,TeslaOwls,2021-09-04 13:28:16.000000,@EconomicManBlog @pauluaq1 @jrtolver16 @Tesla ...,0,en
2,2,vin_wiesel,2021-09-04 13:28:11.000000,@nuc_geek Media: “TESLA CRASHES INTO BUILDING ...,0,en
3,3,Jeeeeee66023780,2021-09-04 13:28:09.000000,@TeslaAIBot @Tesla Why you have so many follow...,0,en
4,4,developrwannab,2021-09-04 13:28:53.000000,@ukspacebulletin Great thanks. Now I got to ad...,0,en


In [39]:
df_drop_index = df.drop(columns = 'index')

In [40]:
df_drop_index.head()

,screen_name,date_time,text,retweeted,lang
0,Zed18978371,2021-09-04 12:48:31.000000,@NorristownN @RolandStautz Yup - I’m the crazy...,0,en
1,TeslaOwls,2021-09-04 13:28:16.000000,@EconomicManBlog @pauluaq1 @jrtolver16 @Tesla ...,0,en
2,vin_wiesel,2021-09-04 13:28:11.000000,@nuc_geek Media: “TESLA CRASHES INTO BUILDING ...,0,en
3,Jeeeeee66023780,2021-09-04 13:28:09.000000,@TeslaAIBot @Tesla Why you have so many follow...,0,en
4,developrwannab,2021-09-04 13:28:53.000000,@ukspacebulletin Great thanks. Now I got to ad...,0,en


In [41]:
len(df_drop_index)

30

In [42]:
# remove duplicates 
df_no_dupes = df_drop_index.drop_duplicates()
df_no_dupes
len(df_no_dupes)

8

In [43]:
# use regex to remove @usernames

df_no_dupes['text'] = df_no_dupes['text'].str.replace('@[^\s]+','')
#re.sub('@[^\s]+','',Tweet)

<ipython-input-43-59960d4ecb67>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_no_dupes['text'] = df_no_dupes['text'].str.replace('@[^\s]+','')
<ipython-input-43-59960d4ecb67>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_dupes['text'] = df_no_dupes['text'].str.replace('@[^\s]+','')


In [44]:
df_no_dupes

,screen_name,date_time,text,retweeted,lang
0,Zed18978371,2021-09-04 12:48:31.000000,Yup - I’m the crazy crypto/Tesla guy..,0,en
1,TeslaOwls,2021-09-04 13:28:16.000000,I don't believe I ever claimed Tesla had ...,0,en
2,vin_wiesel,2021-09-04 13:28:11.000000,Media: “TESLA CRASHES INTO BUILDING WHILE ON ...,0,en
3,Jeeeeee66023780,2021-09-04 13:28:09.000000,Why you have so many followers ?,0,en
4,developrwannab,2021-09-04 13:28:53.000000,Great thanks. Now I got to add Tesla bot 🤔 So...,0,en
5,luke_deluxe,2021-09-04 13:28:44.000000,Fine. Tesla should.,0,en
8,lordhyperkill,2021-09-04 14:52:30.000000,That’s a real thing? Whenever I have heard a...,0,en
9,barryagall,2021-09-04 14:52:29.000000,White interior.,0,en


In [45]:
# use regex to remove punctuation and emojis 
df_no_dupes["text"] = df_no_dupes["text"].str.replace('[^\w\s]','')

<ipython-input-45-0fde4d1c2c14>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_no_dupes["text"] = df_no_dupes["text"].str.replace('[^\w\s]','')
<ipython-input-45-0fde4d1c2c14>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_dupes["text"] = df_no_dupes["text"].str.replace('[^\w\s]','')


In [46]:
df_no_dupes.head()

,screen_name,date_time,text,retweeted,lang
0,Zed18978371,2021-09-04 12:48:31.000000,Yup Im the crazy cryptoTesla guy,0,en
1,TeslaOwls,2021-09-04 13:28:16.000000,I dont believe I ever claimed Tesla had g...,0,en
2,vin_wiesel,2021-09-04 13:28:11.000000,Media TESLA CRASHES INTO BUILDING WHILE ON AU...,0,en
3,Jeeeeee66023780,2021-09-04 13:28:09.000000,Why you have so many followers,0,en
4,developrwannab,2021-09-04 13:28:53.000000,Great thanks Now I got to add Tesla bot So t...,0,en


In [ ]:
# remove stop words 